<a href="https://colab.research.google.com/github/gal-dmitry/TTS_HW/blob/master/FastPitch_Filled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [1]:
!git clone https://github.com/gal-dmitry/TTS_HW
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz


Cloning into 'TTS_HW'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 298 (delta 82), reused 273 (delta 66), pack-reused 0
Receiving objects: 100% (298/298), 5.12 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (82/82), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0  10489      0 --:--:-- --:--:-- --:--:-- 10450
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3287    0  3287    0     0  17484      0 --:--:-- --:--:-- --:--:-- 17484
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [2]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

Cloning into 'apex'...
remote: Enumerating objects: 8764, done.
remote: Counting objects: 100% (1149/1149), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 8764 (delta 1007), reused 879 (delta 867), pack-reused 7615
Receiving objects: 100% (8764/8764), 14.42 MiB | 20.17 MiB/s, done.
Resolving deltas: 100% (5988/5988), done.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Processing /content/apex
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com

In [3]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

  Cloning git://github.com/NVIDIA/dllogger.git (to revision 26a0f8f1958de2c0c460925ff6102a4d2486d6cc) to /tmp/pip-install-pkcl21_e/dllogger_e03fc410a8ba469cac8f7b3a97706ccc
  Running command git clone -q git://github.com/NVIDIA/dllogger.git /tmp/pip-install-pkcl21_e/dllogger_e03fc410a8ba469cac8f7b3a97706ccc
  Running command git rev-parse -q --verify 'sha^26a0f8f1958de2c0c460925ff6102a4d2486d6cc'
  Running command git fetch -q git://github.com/NVIDIA/dllogger.git 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
  Running command git checkout -q 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
     |████████████████████████████████| 183 kB 5.1 MB/s 
     |████████████████████████████████| 195 kB 39.1 MB/s 
  Created wheel for dllogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5630 sha256=2923a4201b17511b0c34db2202834e7994cf7bd038e70c79491a5b4b24e9ecb3
  Stored in directory: /root/.cache/pip/wheels/69/76/a4/ae417dad079ab1cae58eb65601ec01d980c5e02915f4f1792f
  Created wheel for librosa: filename=lib

In [4]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

Extracting nvidia_fastpitch_210824.pt ...
OK
Extracting nvidia_waveglow256pyt_fp16.pt ...
OK


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [ ]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
train.py:43: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
DLL 2021-12-10 14:53:21.580043 - PARAMETER | output :  ./output
DLL 2021-12-10 14:53:21.580351 - PARAMETER | dataset_path :  libri_training_data
DLL 2021-12-10 14:53:21.580441 - PARAMETER | log_file :  ./output/nvlog.json
DLL 2021-12-10 14:53:21.580520 - PARAMETER | pyprof :  False
DLL 2021-12-10 14:53:21.580600 - PARAMETER | epochs :  1000
DLL 2021-12-10 14:53:21.580673 - PARAMETER | epochs_per_

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [ ]:
!cd TTS_HW && bash scripts/inference_example.sh


AMP=false, batch_size=32

DLL 2021-12-10 15:12:04.384648 - PARAMETER | input :  phrases/phrase_1_64.txt
DLL 2021-12-10 15:12:04.384724 - PARAMETER | output :  ./output/audio_phrase_1_64.txt
DLL 2021-12-10 15:12:04.384801 - PARAMETER | log_file :  ./output/audio_phrase_1_64.txt/nvlog_infer.json
DLL 2021-12-10 15:12:04.384884 - PARAMETER | save_mels :  False
DLL 2021-12-10 15:12:04.384960 - PARAMETER | cuda :  True
DLL 2021-12-10 15:12:04.385037 - PARAMETER | cudnn_benchmark :  True
DLL 2021-12-10 15:12:04.385118 - PARAMETER | fastpitch :  output/FastPitch_checkpoint_10.pt
DLL 2021-12-10 15:12:04.385190 - PARAMETER | waveglow :  pretrained_models/waveglow/nvidia_waveglow256pyt_fp16.pt
DLL 2021-12-10 15:12:04.385258 - PARAMETER | sigma_infer :  0.9
DLL 2021-12-10 15:12:04.385325 - PARAMETER | denoising_strength :  0.01
DLL 2021-12-10 15:12:04.385391 - PARAMETER | sampling_rate :  22050
DLL 2021-12-10 15:12:04.385459 - PARAMETER | stft_hop_length :  256
DLL 2021-12-10 15:12:04.385536 - PA

In [ ]:
from IPython.display import Audio, display

In [ ]:
path = "/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav"

In [ ]:
display(Audio(path, autoplay=False)) #1st

In [ ]:
display(Audio(path, autoplay=False)) #2nd

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [41]:
import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from transformers import RobertaTokenizer, RobertaForTokenClassification
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [42]:
def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

###
f1_criterion = torchmetrics.F1(num_classes=3).to(device)
acc_criterion = torchmetrics.Accuracy(num_classes=3).to(device)
###

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.max_len = max_len
        self.tokenizer = tokenizer
        ###
        self.tokenized_sentence = self.tokenizer(sentences, 
                                                 padding=True, 
                                                 truncation=True, 
                                                 max_length=max_len,
                                                 return_tensors="pt")
        ###

    def __getitem__(self, index):
        ###
        label = [val_to_class(value) for value in self.labels[index]]
        mask = self.tokenized_sentence.attention_mask[index]
        ids = self.tokenized_sentence.input_ids[index]
        
        max_size = self.tokenized_sentence.input_ids.shape[-1]
        padding = max_size - len(label)

        if padding > 0:
            label.extend([0] * padding)
        else:
            label = label[:max_size]

        return {
            'input_ids': ids,
            'attention_mask': mask,
            'labels': torch.tensor(label, dtype=torch.long)
        } 
        ###

    def __len__(self):
        return self.len

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    ###
    prom = []
    sent = []
    with open(filename) as file: 
        for line in file: 
            _lst = line.strip().split() 
            _word, _prom = _lst[3], _lst[4]
            _prom = val_to_class(float(_prom))
            prom.append(_prom)
            sent.append(_word)

    sentences.append(" ".join(sentence))
    label_proms.append(prom)
    ###

# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

###
# model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=3)
model = RobertaForTokenClassification.from_pretrained('roberta-base', num_labels=3)
###

model.to(device)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-05)

for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        optimizer.zero_grad()

        ###
        batch = {k: v.to(device) for k, v in batch.items()}
        
        out = model(**batch)
        loss = out.loss
        loss.backward()
        ###

        t_bar.set_description(desc=f"loss {loss.item()}")
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            ###
            batch = {k: v.to(device) for k, v in batch.items()}
            mask = batch["attention_mask"]
            y_true = batch["labels"].masked_select(mask == 1)

            out = model(**batch)
            y_pred = out.logits.argmax(-1)
            y_pred = y_pred.masked_select(mask == 1)
            
            accs.append(acc_criterion(y_pred, y_true))
            f1_scores.append(f1_criterion(y_pred, y_true))
            ###

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")
    print()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

EPOCH:0


loss 0.401847243309021: 100%|██████████| 14/14 [00:04<00:00,  3.09it/s]


Accuracy: 0.8483871221542358
F1 score: 0.8483871221542358

EPOCH:1


loss 0.37708932161331177: 100%|██████████| 14/14 [00:04<00:00,  3.16it/s]


...

Accuracy: 0.8548387289047241
F1 score: 0.8548387289047241

EPOCH:8


loss 0.379675954580307: 100%|██████████| 14/14 [00:04<00:00,  3.14it/s]


Accuracy: 0.8548387289047241
F1 score: 0.8548387289047241

EPOCH:9


loss 0.39771750569343567: 100%|██████████| 14/14 [00:04<00:00,  3.14it/s]

Accuracy: 0.8548387289047241
F1 score: 0.8548387289047241



Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!